# Recherche Predictive - Vision 2026 (XGBoost Synchronise)

Ce notebook est synchronise avec `scripts/train_model.py`. Il utilise un **XGBoost unique** avec une ingénierie de variables de pointe pour atteindre une précision absolue sur les flux d'admissions.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
import joblib
import os

In [ ]:
# 1. Chargement et Preparation des Donnees
df_adm = pd.read_csv('../data/raw/admissions_hopital_pitie_2024.csv')
df_adm['date_entree'] = pd.to_datetime(df_adm['date_entree'])
daily_data = df_adm.groupby('date_entree').size().rename('admissions').reset_index()
daily_data = daily_data.set_index('date_entree').asfreq('D', fill_value=0)

def create_features(df):
    df = df.copy()
    # Temps Cyclique
    df['month_sin'] = np.sin(2 * np.pi * df.index.month / 12)
    df['month_cos'] = np.cos(2 * np.pi * df.index.month / 12)
    df['day_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7)
    df['day_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7)
    
    # Jours Feries et Proximite
    holidays = ['2024-01-01', '2024-04-01', '2024-05-01', '2024-05-08', 
                '2024-05-09', '2024-05-20', '2024-07-14', '2024-08-15', 
                '2024-11-01', '2024-11-11', '2024-12-25']
    holiday_dates = pd.to_datetime(holidays)
    df['is_holiday'] = df.index.strftime('%Y-%m-%d').isin(holidays).astype(int)
    df['days_to_holiday'] = [(holiday_dates[holiday_dates >= d].min() - d).days if any(holiday_dates >= d) else 365 for d in df.index]
    
    # Granularite Temporelle
    df['dayofyear'] = df.index.dayofyear
    df['dayofyear_sin'] = np.sin(2 * np.pi * df['dayofyear'] / 365)
    df['dayofyear_cos'] = np.cos(2 * np.pi * df['dayofyear'] / 365)
    df['weekofyear'] = df.index.isocalendar().week.astype(int)
    df['dayofmonth'] = df.index.day
    
    # Lags Multiples
    df['lag1'] = df['admissions'].shift(1)
    df['lag2'] = df['admissions'].shift(2)
    df['lag7'] = df['admissions'].shift(7)
    df['lag14'] = df['admissions'].shift(14)
    
    # Statistiques Mobiles Avancees
    df['roll_mean_3'] = df['admissions'].shift(1).rolling(window=3).mean()
    df['roll_mean_7'] = df['admissions'].shift(1).rolling(window=7).mean()
    df['roll_max_7'] = df['admissions'].shift(1).rolling(window=7).max()
    df['roll_min_7'] = df['admissions'].shift(1).rolling(window=7).min()
    df['roll_std_7'] = df['admissions'].shift(1).rolling(window=7).std()
    
    return df.dropna()

features_df = create_features(daily_data)
FEATURES = ['month_sin', 'month_cos', 'day_sin', 'day_cos', 'is_holiday', 'days_to_holiday', 
            'dayofyear_sin', 'dayofyear_cos', 'weekofyear', 'dayofmonth', 'lag1', 'lag2', 'lag7', 'lag14', 
            'roll_mean_3', 'roll_mean_7', 'roll_max_7', 'roll_min_7', 'roll_std_7']
TARGET = 'admissions'

train = features_df.iloc[:-30]
test = features_df.iloc[-30:]
X_train, y_train = train[FEATURES], train[TARGET]
X_test, y_test = test[FEATURES], test[TARGET]

## 2. Optimisation via GridSearchCV
Nous utilisons les memes reglages que le script de production pour garantir la coherence.

In [ ]:
param_grid = {
    'n_estimators': [2000],
    'learning_rate': [0.01, 0.05],
    'max_depth': [12, 15],
    'objective': ['reg:squarederror']
}

tscv = TimeSeriesSplit(n_splits=3)
grid = GridSearchCV(xgb.XGBRegressor(), param_grid, cv=tscv, scoring='neg_mean_absolute_error', verbose=1)
grid.fit(X_train, y_train)

print(f"Best Hyperparams found : {grid.best_params_}")
model = grid.best_estimator_

In [ ]:
y_pred = model.predict(X_test)
res = pd.DataFrame({'Reel': y_test, 'Pred': y_pred}, index=test.index)

fig = go.Figure()
fig.add_trace(go.Scatter(x=res.index, y=res['Reel'], name='Reel (Pitie)', line=dict(color='#005ba1', width=3)))
fig.add_trace(go.Scatter(x=res.index, y=res['Pred'], name='XGBoost (Synchronise)', line=dict(color='#c8102e', dash='dot', width=3)))
fig.update_layout(title='Verification de la Coherence Script/Notebook', template='plotly_dark')
fig.show()

print(f"MAE (Decembre) : {mean_absolute_error(y_test, y_pred):.2f}")
print(f"R2 Score (Decembre) : {r2_score(y_test, y_pred):.4f}")